NOTES


API Docs : https://www.prop-odds.com/features
I believe using API, with one call, you can get all markets for one game_id
Core Concept and Logic : 1. Lets start with NFL League 2. Get all Upcoming Games 3. Each game will have multiple Markets ( meaning Moneyline, UnderOver etc refer API Doc for more info) 4. Filter All Markets for each game_id which has bookie_key = 'pinnacle' and ( bookie_key = 'fanduel' and/or bookie_key = 'draftkings' ) 5. I only want markets with abv conditions, do not care about other bookie_keys 6. Each market will have market_key, for every market_key, there is outcome, do group by name, description, handicap, odds, max(timestamp). 7. Using the above logic, i want one row of data, i.e latest timestamp record of the group by

In [77]:
import requests
import urllib
from datetime import datetime

BASE_URL = 'https://api.prop-odds.com'
API_KEY = 'ZkcWPqt4RjHzatSAgz2dPh3ogeXvmLrgArPxIhPCFE'

def get_most_recent_odds(game_id, market):
    query_params = {
        'api_key': API_KEY,
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/odds/' + game_id + '/' + market + '?' + params
    print("get_most_recent_odds",url)
    return get_request(url)

def get_request(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()

    print('Request failed with status:', response.status_code)
    return {}

def get_game_info(game_id):
    query_params = {
        'api_key': API_KEY,
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/game/' + game_id + '?' + params
    print("get_game_info",url)
    return get_request(url)

def get_markets(game_id):
    query_params = {
        'api_key': API_KEY,
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/markets/' + game_id + '?' + params
    print("get_markets",url)
    return get_request(url)

def get_nfl_games():
    url = "https://api.prop-odds.com/beta/games/nfl?date=2024-01-13&tz=America%2FNew_York&api_key=ZkcWPqt4RjHzatSAgz2dPh3ogeXvmLrgArPxIhPCFE"
    return get_request(url)

#Actual Sample Code had below function but
# I do not know how to call all nfl games without date function

"""
def get_nfl_games():
    now = datetime.now()
    query_params = {
        'date': now.strftime('%Y-%m-%d'),
        'tz': 'America/New_York',
        'api_key': API_KEY,
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/games/nba?' + params
    print("get_nfl_games",url)
    return get_request(url)
"""

games = get_nfl_games()
print("Number of games from API Call, ", len(games['games']))


Number of games from API Call,  2


In [78]:
for i in range(0, len(games['games'])):
    first_game = games['games'][i]
    game_id = first_game['game_id']
    print("")
    print("Game_ID",i+1,first_game['game_id'],first_game['away_team'],first_game['home_team'])
    game_info = get_markets(game_id)

#Can custom choose market from list  ; 12 means moneyline
    first_market = game_info['markets'][12]
    odds = get_most_recent_odds(game_id, first_market['name'])


Game_ID 1 46f41f853445f88526a9f3a74436ed46 Cleveland Browns Houston Texans
get_markets https://api.prop-odds.com/beta/markets/46f41f853445f88526a9f3a74436ed46?api_key=ZkcWPqt4RjHzatSAgz2dPh3ogeXvmLrgArPxIhPCFE
get_most_recent_odds https://api.prop-odds.com/beta/odds/46f41f853445f88526a9f3a74436ed46/moneyline?api_key=ZkcWPqt4RjHzatSAgz2dPh3ogeXvmLrgArPxIhPCFE

Game_ID 2 3d21c66ac78070d9c2571230b718b98f Miami Dolphins Kansas City Chiefs
get_markets https://api.prop-odds.com/beta/markets/3d21c66ac78070d9c2571230b718b98f?api_key=ZkcWPqt4RjHzatSAgz2dPh3ogeXvmLrgArPxIhPCFE
get_most_recent_odds https://api.prop-odds.com/beta/odds/3d21c66ac78070d9c2571230b718b98f/moneyline?api_key=ZkcWPqt4RjHzatSAgz2dPh3ogeXvmLrgArPxIhPCFE


In [79]:
# Dictionary of the last game 

print(odds)

{'game_id': '3d21c66ac78070d9c2571230b718b98f', 'sportsbooks': [{'bookie_key': 'fanduel', 'market': {'market_key': 'moneyline', 'outcomes': [{'timestamp': '2024-01-10T00:47:28', 'handicap': 0.0, 'odds': -210, 'participant': None, 'participant_name': None, 'name': 'Kansas City Chiefs', 'description': 'Moneyline'}, {'timestamp': '2024-01-10T00:47:28', 'handicap': 0.0, 'odds': 176, 'participant': None, 'participant_name': None, 'name': 'Miami Dolphins', 'description': 'Moneyline'}]}}, {'bookie_key': 'pinnacle', 'market': {'market_key': 'moneyline', 'outcomes': [{'timestamp': '2024-01-09T23:04:45', 'handicap': 0.0, 'odds': -209, 'participant': None, 'participant_name': None, 'name': 'Kansas City Chiefs', 'description': 'Game Money Line'}, {'timestamp': '2024-01-09T23:04:45', 'handicap': 0.0, 'odds': 181, 'participant': None, 'participant_name': None, 'name': 'Miami Dolphins', 'description': 'Game Money Line'}]}}, {'bookie_key': 'draftkings', 'market': {'market_key': 'moneyline', 'outcomes'

In [80]:
#List of Bookies 
i = 0 
for i in range(5):
    print(odds['sportsbooks'][i]['bookie_key'])

fanduel
pinnacle
draftkings
caesars
betmgm


In [81]:
import pandas as pd
df = pd.DataFrame.from_dict(odds, orient='columns')


In [82]:
df

,game_id,sportsbooks
0,3d21c66ac78070d9c2571230b718b98f,"{'bookie_key': 'fanduel', 'market': {'market_k..."
1,3d21c66ac78070d9c2571230b718b98f,"{'bookie_key': 'pinnacle', 'market': {'market_..."
2,3d21c66ac78070d9c2571230b718b98f,"{'bookie_key': 'draftkings', 'market': {'marke..."
3,3d21c66ac78070d9c2571230b718b98f,"{'bookie_key': 'caesars', 'market': {'market_k..."
4,3d21c66ac78070d9c2571230b718b98f,"{'bookie_key': 'betmgm', 'market': {'market_ke..."
5,3d21c66ac78070d9c2571230b718b98f,"{'bookie_key': 'barstool', 'market': {'market_..."
6,3d21c66ac78070d9c2571230b718b98f,"{'bookie_key': 'betrivers', 'market': {'market..."
7,3d21c66ac78070d9c2571230b718b98f,"{'bookie_key': 'pointsbet', 'market': {'market..."


In [71]:
#Stuck until above ; do not know how to deal with multi level dictionaries

In [84]:
# This can be done for each game id 
#But dont know how to go deeper than this beyond "market.outcomes"
df = pd.json_normalize(odds['sportsbooks'])#.explode('market')
df_ori = df

In [85]:
df_ori

,bookie_key,market.market_key,market.outcomes
0,fanduel,moneyline,"[{'timestamp': '2024-01-10T00:47:28', 'handica..."
1,pinnacle,moneyline,"[{'timestamp': '2024-01-09T23:04:45', 'handica..."
2,draftkings,moneyline,"[{'timestamp': '2024-01-10T00:00:25', 'handica..."
3,caesars,moneyline,"[{'timestamp': '2024-01-09T22:05:11', 'handica..."
4,betmgm,moneyline,"[{'timestamp': '2024-01-09T16:20:28', 'handica..."
5,barstool,moneyline,"[{'timestamp': '2024-01-09T23:06:02', 'handica..."
6,betrivers,moneyline,"[{'timestamp': '2024-01-09T17:31:03', 'handica..."
7,pointsbet,moneyline,"[{'timestamp': '2024-01-09T21:42:06', 'handica..."


In [74]:
odds

{'game_id': '3d21c66ac78070d9c2571230b718b98f',
 'sportsbooks': [{'bookie_key': 'fanduel',
   'market': {'market_key': 'moneyline',
    'outcomes': [{'timestamp': '2024-01-10T00:47:28',
      'handicap': 0.0,
      'odds': -210,
      'participant': None,
      'participant_name': None,
      'name': 'Kansas City Chiefs',
      'description': 'Moneyline'},
     {'timestamp': '2024-01-10T00:47:28',
      'handicap': 0.0,
      'odds': 176,
      'participant': None,
      'participant_name': None,
      'name': 'Miami Dolphins',
      'description': 'Moneyline'}]}},
  {'bookie_key': 'pinnacle',
   'market': {'market_key': 'moneyline',
    'outcomes': [{'timestamp': '2024-01-09T23:04:45',
      'handicap': 0.0,
      'odds': -209,
      'participant': None,
      'participant_name': None,
      'name': 'Kansas City Chiefs',
      'description': 'Game Money Line'},
     {'timestamp': '2024-01-09T23:04:45',
      'handicap': 0.0,
      'odds': 181,
      'participant': None,
      'partici